In [0]:
df=spark.table("dev.riyer_bronze.sales")

df1=df.dropDuplicates().dropna().drop("ingestion_date")

df1.write.mode("overwrite").saveAsTable("dev.riyer_silver.sales_cleaned")

In [0]:
%sql
with deduplicated_source as 
(
  select s.*,
  row_number() over(partition by customer_id order by sequenceNum desc) as row_number
  from dev.riyer_bronze.customers s
  )
merge into dev.riyer_silver.customers t
using (select * from deduplicated_source where row_number =1)s 
on t.customer_id = s.customer_id
when matched and s.operation="DELETE" 
then delete
when matched and s.operation="UPDATE" then update
set 
t.customer_name = s.customer_name,
t.customer_email = s.customer_email,
t.customer_city = s.customer_city,
t.customer_state = s.customer_state
when not matched then insert 
(t.customer_id, t.customer_name, t.customer_email, t.customer_city, t.customer_state) values (s.customer_id, s.customer_name, s.customer_email, s.customer_city, s.customer_state)

In [0]:
%sql
with deduplicated_source as 
(
  select s.*,
  row_number() over(partition by product_id order by seqNum desc) as row_number
  from dev.riyer_bronze.products s
  )
merge into dev.riyer_silver.products t
using (select * from deduplicated_source where row_number =1)s 
on t.product_id = s.product_id
when matched and s.operation="DELETE" 
then delete
when matched and s.operation="UPDATE" then update
set 
t.product_name = s.product_name,
t.product_price = s.product_price,
t.product_category = s.product_category
when not matched then insert 
(t.product_id, t.product_name, t.product_price, t.product_category) values (s.product_id, s.product_name, s.product_price, s.product_category)